# Windows Shell Spawning Suspicious Program
Detects a suspicious child process of a Windows shell

## Rule Content
```
- title: Windows Shell Spawning Suspicious Program
  id: 3a6586ad-127a-4d3b-a677-1e6eacdf8fde
  status: experimental
  description: Detects a suspicious child process of a Windows shell
  references:
  - https://mgreen27.github.io/posts/2018/04/02/DownloadCradle.html
  author: Florian Roth
  date: 2018/04/06
  modified: 2019/02/05
  tags:
  - attack.execution
  - attack.defense_evasion
  - attack.t1064
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage:
      - '*\mshta.exe'
      - '*\powershell.exe'
      - '*\rundll32.exe'
      - '*\cscript.exe'
      - '*\wscript.exe'
      - '*\wmiprvse.exe'
      Image:
      - '*\schtasks.exe'
      - '*\nslookup.exe'
      - '*\certutil.exe'
      - '*\bitsadmin.exe'
      - '*\mshta.exe'
    falsepositives:
      CurrentDirectory: '*\ccmcache\\*'
    condition: selection and not falsepositives
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Administrative scripts
  - Microsoft SCCM
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:(*\\mshta.exe OR *\\powershell.exe OR *\\rundll32.exe OR *\\cscript.exe OR *\\wscript.exe OR *\\wmiprvse.exe) AND process_path.keyword:(*\\schtasks.exe OR *\\nslookup.exe OR *\\certutil.exe OR *\\bitsadmin.exe OR *\\mshta.exe)) AND (NOT (process_current_directory.keyword:*\\ccmcache\\*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()